In [46]:
%load_ext autoreload
%autoreload 2

from dask_image.ndfilters import median_filter, threshold_local, percentile_filter
from dask_image.ndfilters import convolve, maximum_filter
from pyfires.PYF_WindowStats import get_mea_std_window
from cython.parallel import prange

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from datetime import datetime
from satpy import Scene
import dask.array as da
from glob import glob
from pyfires.PYF_detection import stage1_tests, run_basic_night_detection

from pyfires.PYF_detection import stage1_tests, run_basic_night_detection, _make_kern
import pyfires.PYF_Consts as PYFc
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from satpy import Scene
from glob import glob
import os
bdict = {'vi1_band': 'B03',
         'vi2_band': 'B06',
         'mir_band': 'B07',
         'lwi_band': 'B13'}


curfile = 'D:/sat_data/ahi_main/in/0840/HS_H09_20231029_0840_B07_FLDK_R20_S0110.DAT'
tod = 'D:/sat_data/ahi_main/out/'
pos = curfile.find('B07')
dtstr = curfile[pos-14:pos-1]
ifiles_l15 = glob(f'{os.path.dirname(curfile)}/*{dtstr}*.DAT')
in_data_dict = initial_load(ifiles_l15,  # List of files to load
                            'ahi_hsd',   # The reader to use, in this case the AHI HSD reader
                            bdict,       # The bands to load
                            mir_bt_thresh=270, # The threshold for the MIR band, pixels cooler than this are excluded.
                            lw_bt_thresh=260)

#scn.save_datasets(base_dir=tod, enhance=False, dtype=np.float32)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


cannot convert float NaN to integer


In [43]:
pfp = stage1_tests(in_data_dict['MIR__BT'],
                   in_data_dict['BTD'],
                   in_data_dict['VI1_DIFF'],
                   in_data_dict['SZA'],
                   in_data_dict['LSM'],
                   ksizes=[5, 7, 9],
                   do_lsm_mask=True)

opts = {'def_fire_rad_vis': 0.5,
        'def_fire_rad_vid': 0.1,
        'sza_thresh': 97,
        'vid_thresh': 0.02
        }

#scn.save_dataset('PFP', enhance=False, dtype=np.float32, base_dir=tod)

night_res = run_basic_night_detection(in_data_dict['VI2_RAD'],
                                      in_data_dict['SZA'],
                                      in_data_dict['VI1_DIFF'],
                                      pfp,
                                      opts)

In [3]:
vid_arr = scn['VI1_DIFF'].data
sza_arr = scn['SZA'].data
lwi_arr = scn['LW1__BT'].data

vid_arr_np = np.array(vid_arr)
lwi_arr_np = np.array(lwi_arr)
sza_arr_np = np.array(sza_arr)

In [4]:
#scn.save_dataset('SZA', enhance=False, dtype=np.float32, base_dir=tod)

In [30]:

#scn = vid_adjust_sza(scn)
#scn.save_dataset('VI1_DIFF_2', base_dir=tod, enhance=False, dtype=np.float32)

In [6]:
dets = da.where(scn['VI1_DIFF_2'] > 0, 1, 0)
dets = da.where(scn['PFP'] > 0, dets, 0)

In [7]:
#save_output(scn, dets, 'dets', tod, 'VI1_DIFF')

In [8]:
#scn.save_dataset('VI1_DIFF_2', base_dir=tod, enhance=False, dtype=np.float32)
#scn.save_dataset('VI1_DIFF', base_dir=tod, enhance=False, dtype=np.float32)

In [9]:
dets_arr = np.array(dets)
vi_rad_arr = np.array(scn['VI1_RAD'])
ndfi_arr = np.array(scn['mi_ndfi'])
vi_diff_arr = np.array(scn['VI1_DIFF'])
btd_arr = np.array(scn['BTD']) 
mir_bt_arr = np.array(scn['MIR__BT'])
irr_arr = np.array(scn['VI2_RAD'])
lwi_bt_arr = np.array(scn['LW1__BT'])
lwi_rad_arr = np.array(scn['LW1_RAD'])
lsm = np.array(scn['LSM'])
sza_arr = scn['SZA'].data

In [10]:
dets_arr = np.array(dets).astype(np.uint8)

In [11]:
st_time = datetime.utcnow()
outa = get_mea_std_window(dets_arr,      # Potential fire pixel candidates
                          vi_rad_arr,    # VIS chan
                          ndfi_arr,      # NDFI
                          lwi_bt_arr,    # LW Brightness Temperature
                          btd_arr,       # MIR-LW BTD
                          mir_bt_arr,    # MIR BT
                          vi_diff_arr,   # MIR-LWIR-VIS radiance diff
                          lsm,           # The land-sea mask
                          255,
                          15)
en_time = datetime.utcnow()

print((en_time - st_time).total_seconds())

0.763708


In [12]:
perc_good  = np.squeeze(outa[0,:,:]); n_winpix   = np.squeeze(outa[1,:,:])
n_cloudpix = np.squeeze(outa[2,:,:]); n_waterpix = np.squeeze(outa[3,:,:])
mean_lw    = np.squeeze(outa[4,:,:]); std_lw     = np.squeeze(outa[5,:,:])
mean_nd    = np.squeeze(outa[6,:,:]); std_nd     = np.squeeze(outa[7,:,:])
mean_vi    = np.squeeze(outa[8,:,:]); std_vi     = np.squeeze(outa[9,:,:])
mean_btd   = np.squeeze(outa[10,:,:]); std_btd    = np.squeeze(outa[11,:,:])
mean_mir   = np.squeeze(outa[12,:,:]); std_mir    = np.squeeze(outa[13,:,:])
mean_vid   = np.squeeze(outa[14,:,:]); std_vid    = np.squeeze(outa[15,:,:])

vis_window_arr = vi_rad_arr - (mean_vi + 1.5 * std_vi)
ndfi_window_arr = ndfi_arr - (mean_nd + 1.5 * std_nd)
btd_window_arr = btd_arr - (mean_btd + 1.5 * std_btd)
mir_window_arr = mir_bt_arr - (mean_mir + 1.5 * std_mir)
visdif_window_arr = vi_diff_arr - (mean_vid + 1.5 * std_vid)

x1 = 0.02
x2 = 8.

t1 = 270
t2 = 300

grad = (x2 - x1) / (t2 - t1)

lw_std_mult = grad * (lwi_bt_arr - t1)
lw_std_mult = np.where(lw_std_mult > x2, x2, lw_std_mult)
lw_std_mult = np.where(lw_std_mult < x1, x1, lw_std_mult)

lw_window_arr =  mean_lw - lw_std_mult * std_lw

In [13]:
save_output(scn, mean_nd, 'mean_nd', tod, 'VI1_DIFF')
save_output(scn, std_nd, 'std_nd', tod, 'VI1_DIFF')
save_output(scn, mean_lw, 'mean_lw', tod, 'VI1_DIFF')
save_output(scn, std_lw, 'std_lw', tod, 'VI1_DIFF')
save_output(scn, mean_btd, 'mean_btd', tod, 'VI1_DIFF')
save_output(scn, std_btd, 'std_btd', tod, 'VI1_DIFF')
save_output(scn, mean_vid, 'mean_vid', tod, 'VI1_DIFF')
save_output(scn, std_vid, 'std_vid', tod, 'VI1_DIFF')
save_output(scn, mean_mir, 'mean_mir', tod, 'VI1_DIFF')
save_output(scn, std_mir, 'std_mir', tod, 'VI1_DIFF')

#save_output(scn, perc_good, 'perc_good', tod, 'VI1_DIFF')
save_output(scn, n_cloudpix, 'n_cloudpix', tod, 'VI1_DIFF')
save_output(scn, n_waterpix, 'n_waterpix', tod, 'VI1_DIFF')
save_output(scn, n_winpix, 'n_winpix', tod, 'VI1_DIFF')

In [14]:
gd_t4 = mean_mir + 2 * std_mir
gd_btd1 = mean_btd + 2.5
gd_btd2 = mean_btd + 2 * std_btd

btd_thresh = np.where(sza_arr > 70, 3, 5)

lwbt_mult = (lwi_bt_arr - 270) / 30
lwbt_mult = np.where(lwbt_mult > 1, 1, lwbt_mult)
lwbt_mult = np.where(lwbt_mult < 0, 0, lwbt_mult)
lwbt_mult = 1. / (0.25 + 0.75 * lwbt_mult)

sza_mult = np.cos(np.deg2rad(sza_arr))
sza_mult = 0.4 + ((90 - sza_arr) / 90.)
ndfi_thresh = 0.001 + ((0.006 * lwbt_mult) + (0.003 / perc_good)) * sza_mult

#save_output(scn, gd_t4, 'gd_t4', tod, 'VI1_DIFF')
#save_output(scn, gd_btd1, 'gd_btd1', tod, 'VI1_DIFF')
#save_output(scn, gd_btd2, 'gd_btd2', tod, 'VI1_DIFF')
#save_output(scn, lwbt_mult, 'lwbt_mult', tod, 'VI1_DIFF')
#save_output(scn, ndfi_thresh, 'ndfi_thresh', tod, 'VI1_DIFF')
#save_output(scn, sza_mult, 'sza_mult', tod, 'VI1_DIFF')

In [15]:
outarr = da.where(dets_arr > 0,                  1,        0)      #t1
outarr = da.where(ndfi_window_arr > ndfi_thresh, outarr+1, outarr) #t2
outarr = da.where(visdif_window_arr > 0,         outarr+1, outarr) #t3
outarr = da.where(vis_window_arr < 0.1,          outarr+1, outarr) #t4
outarr = da.where(lwi_bt_arr > 260,              outarr+1, outarr) #t5
outarr = da.where(mir_window_arr > 0,            outarr+1, outarr) #t6
outarr = da.where(lsm == 2,                      outarr+1, outarr) #t7
outarr = da.where(mir_bt_arr > gd_t4,            outarr+1, outarr) #t8
outarr = da.where(btd_arr > gd_btd1,             outarr+1, outarr) #t9
outarr = da.where(btd_arr > gd_btd2,             outarr+1, outarr) #t10
outarr = da.where(lwi_bt_arr > lw_window_arr,    outarr+1, outarr) #t11
outarr = da.where(perc_good > 0.4,               outarr,   0)      #t12
outarr = da.where(dets_arr > 0,                  outarr,   0)      #final

vi_diff_def_thr = 0.25

outarr = da.where(vi_diff_arr > vi_diff_def_thr, 10, outarr)

save_output(scn, outarr, 'outarr', tod, 'VI1_DIFF')

In [55]:
conf_val = do_stage5(btd_arr, mean_btd, std_btd, mir_bt_arr, mean_mir, std_mir, sza_arr, n_winpix, n_cloudpix, n_waterpix)
conf_val = np.where(outarr > 0, conf_val, 0)
save_output(scn, conf_val, 'conf_val', tod, 'VI1_DIFF')

In [56]:
a_val = PYFc.rad_to_bt_dict[scn['pix_area'].attrs['platform_name']]
frp_est = (scn['pix_area'] * PYFc.sigma / a_val) * (scn['MIR__BT'] - mean_mir)
frp_est = np.where(mean_mir > 0, frp_est, 0)

scn['frp_est'] = scn['VI1_DIFF'].copy()
scn['frp_est'].attrs['name'] = 'frp_estimate'
scn['frp_est'].attrs['units'] = 'MW'
scn['frp_est'].data = frp_est


In [58]:
results = np.where(outarr >= 10, frp_est, 0)

In [59]:

save_output(scn, results, 'results', tod, 'VI1_DIFF')

In [3]:
scn = Scene(ifiles_l15, reader='ahi_hsd')
scn.load(['true_color_reproduction', 'B13'])
scn = scn.resample(scn.coarsest_area(), resampler='gradient_search')
scn.save_dataset('true_color_reproduction', base_dir=tod)

The following datasets were not created and may require resampling to be generated: DataID(name='true_color_reproduction')
The input area chunks are large. This usually means that the input area is of much higher resolution than the output area. You can reduce the chunks passed, and ponder whether you are using the right resampler for the job.
The input area chunks are large. This usually means that the input area is of much higher resolution than the output area. You can reduce the chunks passed, and ponder whether you are using the right resampler for the job.
The input area chunks are large. This usually means that the input area is of much higher resolution than the output area. You can reduce the chunks passed, and ponder whether you are using the right resampler for the job.
The input area chunks are large. This usually means that the input area is of much higher resolution than the output area. You can reduce the chunks passed, and ponder whether you are using the right resample

In [57]:
from Example_Detection import main

ds_di = main(curfile, tod)

cannot convert float NaN to integer


In [52]:
bob = make_output_scene(in_data_dict)

In [60]:
bob._datasets.keys()

[DataID(name='BTD', wavelength=WavelengthRange(min=3.7, central=3.9, max=4.1, unit='µm'), resolution=2000, calibration=<3>, modifiers=()),
 DataID(name='LSM', wavelength=WavelengthRange(min=3.7, central=3.9, max=4.1, unit='µm'), resolution=2000, calibration=<3>, modifiers=()),
 DataID(name='LW1_RAD', wavelength=WavelengthRange(min=10.2, central=10.4, max=10.6, unit='µm'), resolution=2000, calibration=<3>, modifiers=()),
 DataID(name='LW1__BT', wavelength=WavelengthRange(min=10.2, central=10.4, max=10.6, unit='µm'), resolution=2000, calibration=<2>, modifiers=()),
 DataID(name='MIR_RAD', wavelength=WavelengthRange(min=3.7, central=3.9, max=4.1, unit='µm'), resolution=2000, calibration=<3>, modifiers=()),
 DataID(name='MIR_RAD_NO_IR', wavelength=WavelengthRange(min=3.7, central=3.9, max=4.1, unit='µm'), resolution=2000, calibration=<3>, modifiers=()),
 DataID(name='MIR__BT', wavelength=WavelengthRange(min=3.7, central=3.9, max=4.1, unit='µm'), resolution=2000, calibration=<2>, modifiers=

In [78]:
np.array((scn['bob'] > 240)  + (scn['bob'] < 240)) 

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [75]:
np.nanmax((scn['bob'] > 240).astype(np.uint8))

1

In [76]:
np.nanmin(scn['bob'])

0